In [36]:
import time
import random
import requests
import joblib
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [4]:
data = joblib.load("../data/sp500_top100_v4.bin")
data.head()

,title,date,query,url,ticker,content,ps_content,ps_content_filter,sim
0,BRIEF-Apple Inc Says Not Allowing Entertainmen...,2020-03-15,Apple Inc.,https://www.reuters.com/article/idUSFWN2B61K2,AAPL,March 14 (Reuters) - Apple Inc: * APPLE INC SA...,,empty content,1.0
1,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSKBN20Z33J,AAPL,(Reuters) - Apple Inc has sealed multi-year li...,(Reuters) - Apple Inc has sealed multi-year li...,(Reuters) - Apple Inc has sealed multi-year li...,1.0
2,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSL4N2B54T2,AAPL,(Reuters) - Apple Inc has sealed multi-year li...,NaN,empty content,1.0
3,Chinese regulators remove 'Plague Inc' game fr...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSKCN20M043,AAPL,BEIJING/SHANGHAI (Reuters) - The video game “P...,The regulator did not respond to Reuters phone...,The regulator did not respond to Reuters phone...,1.0
4,UPDATE 1-Chinese regulators remove 'Plague Inc...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSL3N2AS0OO,AAPL,BEIJING/SHANGHAI (Reuters) - The video game “P...,NaN,empty content,1.0


In [37]:
def get_content(url):
    res = requests.get(url, timeout=10)
    soup = BeautifulSoup(res.text, "html.parser")
    paragraph = []
    for element in soup.find_all("p"):
        paragraph.append("".join(element.find_all(text=True)))
    time.sleep(random.randint(5, 10))
    return "".join(paragraph[1:-2])

In [23]:
sp500_top20_company = [
    "AAPL", "MSFT", "AMZN", "FB", "GOOG", "JNJ", "BRK-B", "PG", "V", "JPM", 
    "UNH", "HD", "MA", "NVDA", "VZ", "PYPL", "NFLX", "ADBE", "PFE", "T"]

In [39]:
new_data = pd.DataFrame()
for ticker in sp500_top20_company:
    new_data = pd.concat([new_data, data[data["ticker"] == ticker]], axis=0)

In [40]:
new_data = new_data.iloc[:, 0:5]
new_data

,title,date,query,url,ticker
0,BRIEF-Apple Inc Says Not Allowing Entertainmen...,2020-03-15,Apple Inc.,https://www.reuters.com/article/idUSFWN2B61K2,AAPL
1,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSKBN20Z33J,AAPL
2,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSL4N2B54T2,AAPL
3,Chinese regulators remove 'Plague Inc' game fr...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSKCN20M043,AAPL
4,UPDATE 1-Chinese regulators remove 'Plague Inc...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSL3N2AS0OO,AAPL
...,...,...,...,...,...
126579,Deutsche Telekom CEO denies T-Mobile/Sprint de...,2019-12-10,AT%26T+Inc.,https://www.reuters.com/article/idUSKBN1YE2I3,T
126580,Nikkei rises as SoftBank Group soars on Sprint...,2020-02-12,AT%26T+Inc.,https://www.reuters.com/article/idUSL4N2AC1Q3,T
126581,REFILE-BRIEF-AT&T Expects 2020 Dividend Payout...,2020-06-18,AT%26T+Inc.,https://www.reuters.com/article/idUSFWN2DU0XL,T
126583,Japan Inc shifts to online press briefings as ...,2020-02-25,AT%26T+Inc.,https://www.reuters.com/article/idUSKCN20J17S,T


In [ ]:
tqdm.pandas()
top_20_data = pd.DataFrame()
for ticker in sp500_top20_company:
    new_data["content"] = new_data[new_data["ticker"] == ticker]["url"].progress_apply(lambda row: get_content(row))
    top_20_data = pd.concat([top_20_data, new_data])
    time.sleep(random.randint(20, 30))